In [1]:
import openpyxl as opxl
import pandas as pd
import re
import numpy as np
import unicodedata
from IPython.display import display, HTML

In [2]:
def convert_to_lower_case(data):
    if type(data) is dict:
        for k, v in data.iteritems():
            if type(v) is str:
                data[k] = v.lower()
            elif type(v) is list:
                data[k] = [x.lower() for x in v]
            elif type(v) is dict:
                data[k] = convert_to_lower_case(v)
    return data

In [3]:
files_to_read = [
    {
        'output_file' : '2014-15_gems_jade', \
        'year': '2014-2015', \
        'xls_file' : 'Annex 18 - Reconciliation sheets Gems & Jade 14-15.xlsx', \
        'name_cols': 'B:D', \
        'data_cols' : 'A:M', \
        'data_skip_rows' : 4, \
        'stop_at' : '1000'
    },
    {
        'output_file' : '2014-15_oil_gas', \
        'year': '2014-2015', \
        'xls_file' : 'Annex 18 - Reconciliation sheets Oil & Gas 14-15.xlsx', \
        'name_cols': 'B:E', \
        'data_cols' : 'A:M', \
        'data_skip_rows' : 5, \
        'stop_at' : '1000'
    },
    {
        'output_file' : '2014-15_oil_gas_transport', \
        'year': '2014-2015', \
        'xls_file' : 'Annex 18 - Reconciliation sheets Oil & Gas transp 15-16.xlsx', \
        'name_cols': 'B:E', \
        'data_cols' : 'A:M', \
        'data_skip_rows' : 5, \
        'stop_at' : '1000'
    },
    {
        'output_file' : '2014-15_other_minerals', \
        'year': '2014-2015', \
        'xls_file' : 'Annex 18 - Reconciliation sheets Other minerals 14-15.xlsx', \
        'name_cols': 'B:E', \
        'data_cols' : 'A:M', \
        'data_skip_rows' : 5, \
        'stop_at' : '1000'
    },
    {
        'output_file' : '2015-16_gems_jade', \
        'year': '2015-2016', \
        'xls_file' : 'Annex 18 - Reconciliation sheets Gems & Jade 15-16.xlsx', \
        'name_cols': 'C:E', \
        'data_cols' : 'A:N', \
        'data_skip_rows' : 4, \
        'stop_at' : '51'
    },
    {
        'output_file' : '2015-16_oil_gas', \
        'year': '2015-2016', \
        'xls_file' : 'Annex 18 - Reconciliation sheets Oil & Gas 15-16.xlsx', \
        'name_cols': 'B:E', \
        'data_cols' : 'A:M', \
        'data_skip_rows' : 5, \
        'stop_at' : '35'
    },
    {
        'output_file' : '2015-16_oil_gas_transport', \
        'year': '2015-2016', \
        'xls_file' : 'Annex 18 - Reconciliation sheets Oil & Gas transp 15-16.xlsx', \
        'name_cols': 'B:E', \
        'data_cols' : 'A:M', \
        'data_skip_rows' : 5, \
        'stop_at' : '5'
    },
    {
        'output_file' : '2015-16_other_minerals', \
        'year': '2015-2016', \
        'xls_file' : 'Annex 18 - Reconciliation sheets Other minerals 15-16.xlsx', \
        'name_cols': 'B:E', \
        'data_cols' : 'A:M', \
        'data_skip_rows' : 5, \
        'stop_at' : '28'
    }
]

'''
files_to_read = [
    {
        'output_file' : '2015-16_other_minerals', \
        'year': '2015-2016', \
        'xls_file' : 'Annex 18 - Reconciliation sheets Other minerals 15-16.xlsx', \
        'name_cols': 'B:E', \
        'data_cols' : 'A:M', \
        'data_skip_rows' : 5, \
        'stop_at' : '28'
    }
]

'''

#name_cols = 'B:D'
#data_cols = 'A:M'
#data_skip_rows = 4

"\nfiles_to_read = [\n    {\n        'output_file' : '2015-16_other_minerals',         'year': '2015-2016',         'xls_file' : 'Annex 18 - Reconciliation sheets Other minerals 15-16.xlsx',         'name_cols': 'B:E',         'data_cols' : 'A:M',         'data_skip_rows' : 5,         'stop_at' : '28'\n    }\n]\n\n"

In [4]:
df_all = pd.DataFrame({'company_name' : [],'name_of_revenue_stream' : [], \
                       'paid_to' : [], 'payment_category' : [], 'units' : [], \
                       'per_company_original' : [], \
                       'per_company_adjust' : [], \
                       'per_company_final' : [], \
                       'per_government_original' : [], \
                       'per_government_adjust' : [], \
                       'per_government_final' : [], \
                       'final_difference' : [], \
                       'comment' : []
                      })

key_terms = { 'payment_category' : ['Payments in kind', 'Payments in cash', 'B- Unilateral company disclosures'], \
              'units' : ['In Barils', 'In Mscf', 'Gold in T.oz', 'Tin in MT', 'In (Please mention the commodity)', \
                       'Antimony Ore', 'NA', 'Copper', 'Copper in MT', 'Ferro Nickel']
            }
key_terms = convert_to_lower_case(key_terms)

In [5]:
def join_column_titles(text):
    #print(text)
    to_return = ""
    for t in text:
        #print(t)
        if not isinstance(t, float):
            to_return = '.'.join([to_return,(''.join(i for i in t if ord(i)<128))])
    #print("FINAL: " + to_return)
    
    # remove first '.' from the title
    return to_return[1:]

In [6]:
def rename_duplicate_column_titles(columns):
    unique_titles = []
    title_counts = {}
    
    for c in columns:
        if c in unique_titles:
            title_counts[c] += 1
            unique_titles.append(c + "." + str(title_counts[c]))
        else:
            title_counts[c] = 0
            unique_titles.append(c)
            
    return unique_titles

In [7]:
def add_sheet_to_main_df(main_df,current_df, company_name,key_terms):

    
    current_payment_category = ""
    current_units = "MMK"
    current_paid_to = ""

            
    for index, row in current_df.iterrows():
        
        #display(row)
            
        index_col = 'n'
        description_col = 'description of payment'
        
        # skip row if '°' is included in it because that means 
        #  there's an extra row of French titles in the table
        unicode_row = row.to_string().encode("utf-8")        
        if u'\xb0' in unicode_row.decode('windows-1252'):
            continue
        
        # if the index column in empty, that means it's not a data-row
        if str(row[index_col])  == 'nan':
            index_col = description_col
        
        #print(str(row[description_col]).lower() + " | " + str(row[index_col]).lower())
        
        if str(row[index_col]).lower() in key_terms['payment_category']:
            current_payment_category = str(row[index_col]).lower()
        elif str(row[index_col]).lower() in key_terms['units']:
            current_units = str(row[index_col]).lower()
            current_paid_to = ""
        elif not str(row[index_col]).replace('.','',1).isdigit():
            current_paid_to = str(row[index_col])
            current_units = "MMK"
        
        if str(row['n']).replace('.','',1).isdigit():
            to_append = pd.DataFrame({'company_name' : [company_name], \
                                      'name_of_revenue_stream' : [row['description of payment']] , \
                                      'paid_to' : [current_paid_to], \
                                      'payment_category' : [current_payment_category], \
                                      'units' : [current_units], \
                                      'per_company_original' : [row['per company.original']], \
                                      'per_company_adjust' : [row['per company.adjust']], \
                                      'per_company_final' : [row['per company.final']], \
                                      'per_government_original' : [row['per government.original']], \
                                      'per_government_adjust' : [row['per government.adjust']], \
                                      'per_government_final' : [row['per government.final']], \
                                      'final_difference' : [row['final difference']], \
                                      'comment' : [row['comment']] \
                                    })
            
            
            to_append['comment'].fillna('', inplace=True)
            to_append.fillna(0, inplace=True)
            
            main_df = pd.concat([main_df, to_append])
        
    return main_df
    
    

In [8]:
def read_files(files_to_read, df_all): 
    for f in files_to_read:

        output_file = f['output_file']
        year = f['year']
        xls_file = f['xls_file']
        name_cols = f['name_cols']
        data_cols = f['data_cols']
        data_skip_rows = f['data_skip_rows']   
        stop_at = f['stop_at']       

        xl = pd.ExcelFile(year+'/'+xls_file)
#        '2014-2015/Annex 18 - Reconciliation sheets Gems & Jade 14-15.xlsx')


        sheets_list = xl.sheet_names
    
        print(sheets_list)


        for s in sheets_list:
            
            if not re.search("[A-Z]+\s\(\d+\)",s):
                continue
            else:
                match= re.search(r'(\d+)', s)
                print('current sheet number: ' + match.group(1))
                if int(match.group(1)) > int(stop_at):
                    continue

            #print(s)
            sheet_name = s

            name_df = xl.parse(sheet_name, parse_cols = name_cols)
            #display(name_df.head())

            # xl.parse takes 1st ro parsed into column headers - so we retrieve company name
            #  from the name of the column
            company_name = name_df.columns[2]
            #print(company_name)

            # xl.parse takes 1st ro parsed into column headers - so we retrieve regsitry number
            #  from the row where "Company name:" column is equal to "Registry number"
            if 'Company name:' in name_df.columns:
                company_number = name_df[name_df['Company name:'] == 'Registry number'][company_name]
            else:
                company_number = name_df[name_df['Company name'] == 'Registry number'][company_name]

            #print(company_name, company_number)

            df = xl.parse(sheet_name, skiprows=data_skip_rows, parse_cols = data_cols, header=None)        
            
            
            for i in df.iloc[0:1]:
                if df.iloc[0:1][i][0] == 'Company':
                    df.iloc[0:1][i][0] = 'per company'
                if df.iloc[0:1][i][0] == 'Government Agency':
                    df.iloc[0:1][i][0] = 'per government'
                    
            
            # fill out cells with merged column headers
            df.iloc[0:2] = df.iloc[0:2].fillna(method='ffill', axis=1)
            df.columns = df.iloc[0:2].apply(join_column_titles, axis=0)
            df = df.iloc[2:]
            df = df.reset_index(drop=True)
            df.columns = rename_duplicate_column_titles(df.columns)
            df.columns = [x.lower() for x in df.columns]
            df.rename(columns={'final difference.final': 'final difference', \
                               'comment.final': 'comment'}, inplace=True)
            if 'n.n' in df.columns:
                df.rename(columns={'n.n': 'n'}, inplace=True) 
            if 'description of payment.description' in df.columns:
                df.rename(columns={'description of payment.description': \
                                   'description of payment'}, inplace=True)
                
            for col in df.columns:
                new_col_name = col.replace("governement", "government")
                new_col_name = new_col_name.replace("ajust", "adjust")
                df.rename(columns={col: new_col_name}, inplace=True)
            
            if 'per company.initial' in df.columns:
                df.rename(columns={'per company.initial': \
                                   'per company.original'}, inplace=True)
            if 'per government.initial' in df.columns:
                df.rename(columns={'per government.initial': \
                                   'per government.original'}, inplace=True)
                
                
            if 'company.initial' in df.columns:
                df.rename(columns={'company.initial': \
                                   'per company.original'}, inplace=True)
            if 'government agency.initial' in df.columns:
                df.rename(columns={'government agency.initial': \
                                   'per government.original'}, inplace=True)


            #display(df.head())
            # clean name for the company to save individual CSV file if needed
            name = map(lambda x: ''.join(e for e in x if e.isalnum()) , company_name.split(' '))
            name = ' '.join(w for w in name)
            name = re.sub( '\s+', ' ', name.strip())
            filename = re.sub( '\s+', '_', name.strip())+'.csv'
            #         df.to_csv(filename)

            print(filename)

            df_all = add_sheet_to_main_df(df_all,df, company_name,key_terms)
    
        df_all = df_all.reset_index(drop=True)
        df_all.to_csv(year+'/'+output_file+'.csv', encoding='utf-8')

        display(df_all.head())


In [9]:
read_files(files_to_read,df_all)

IOError: [Errno 2] No such file or directory: '2014-2015/Annex 18 - Reconciliation sheets Gems & Jade 14-15.xlsx'